In [1]:
import requests, sqlite3
from sqlite3 import Error

In [2]:
inventory_sheet_url = 'https://docs.google.com/spreadsheets/d/1abfWjKouFxD_erzhdnam8tCtFC5G_956/edit#gid=2096778584'
schedule_sheet_url = 'https://docs.google.com/spreadsheets/d/19jmopenGv7VnXOWeBmPpsicmCZycEfOl/edit#gid=2051886986'
pnr_sheet_url = 'https://docs.google.com/spreadsheets/d/1E3Vnx5WA0ntOG42A_oyNI1VCNMn_UY6938mt9Mxz2gs/edit#gid=0'

In [3]:
def get_connection(db_file):
    """Create a database connection to a SQLite database"""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connection to the '{db_file}' SQLite database successful!")
    except Error as e:
        print(f"Something went wrong while connecting to the '{db_file}' SQLite database!\nError: {e}")

    return conn

In [4]:
def get_sheet_data(sheet_url):
    """Returns a 2-dimensional array of data from the given sheet"""
    sheet_id = sheet_url.split('/')[5]
    gid = sheet_url.split('/')[6].split('=')[1]
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=tsv&gid={gid}"
    response = requests.get(url)
    data = response.text
    rows = data.split('\n')
    sheet_data = []
    for row in rows:
        sheet_data.append(row.split('\t'))
    return sheet_data

In [5]:
def parse_date(date):
    """MM/DD/YYYY to YYYY-MM-DD"""
    m, d, y = date.split('/')
    return f"{y}-{m}-{d}"

def parse_date_time(datetime):
    """MM/DD/YYYY HH:MM:SS to YYYY-MM-DD HH:MM:SS"""
    date, time = datetime.split(' ')
    return parse_date(date) + ' ' + time

In [17]:
conn = get_connection('db.sqlite3')
cur = conn.cursor()

Connection to the 'db.sqlite3' SQLite database successful!


In [7]:
inventory_data = get_sheet_data(inventory_sheet_url)

In [18]:
for row in inventory_data[1:]:
    inventory_id = row[0]
    schedule_id = row[1]
    flight_number = row[2]
    aircraft_type = row[3]
    departure_date = parse_date(row[4])
    arrival_date = parse_date(row[5])
    total_capacity = row[8]
    total_inventory = row[9]
    booked_inventory = row[10]
    
    cur.execute(
        f"""
        INSERT INTO inventory VALUES (
            '{inventory_id}',
            '{schedule_id}',
            '{flight_number}',
            '{aircraft_type}',
            '{departure_date}',
            '{arrival_date}',
            {total_capacity},
            {total_inventory},
            {booked_inventory}
        );
        """
    )
    conn.commit()

In [10]:
schedule_data = get_sheet_data(schedule_sheet_url)

In [19]:
for row in schedule_data[1:]:
    schedule_id = row[0]
    flight_number = row[2]
    aircraft_type = row[3]
    aircraft_tail_number = row[4]
    departure_airport = row[5]
    arrival_airport = row[6]
    departure_time = row[7]
    arrival_time = row[8]
    start_date = row[9]
    end_date = row[10]
    status = row[11]
    schedule_frequency = ''.join(row[12:19])
    
    cur.execute(
        f"""
        INSERT INTO Schedule VALUES (
            '{schedule_id}',
            '{flight_number}',
            '{aircraft_type}',
            '{aircraft_tail_number}',
            '{departure_airport}',
            '{arrival_airport}',
            '{departure_time}',
            '{arrival_time}',
            '{start_date}',
            '{end_date}',
            '{status}',
            '{schedule_frequency}'
        );
        """
    )
    conn.commit()

In [12]:
pnr_data = get_sheet_data(pnr_sheet_url)

In [20]:
for row in pnr_data[1:]:
    passenger_id = row[0]
    flight_number = row[1]
    departure_date = parse_date(row[2])
    arrival_date = parse_date(row[3])
    departure_time = row[4]
    arrival_time = row[5]
    is_paid = row[6]
    cabin_class = row[7]
    
    cur.execute(
        f"""
        INSERT INTO PNR VALUES (
            '{passenger_id}',
            '{flight_number}',
            '{departure_date}',
            '{arrival_date}',
            '{departure_time}',
            '{arrival_time}',
            '{is_paid}',
            '{cabin_class}'
        );
        """
    )
    conn.commit()